In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
import re
import pickle
import modules.utils as Utils
from modules.regex_patterns import reviews_paths
from collections import Counter
from underthesea import word_tokenize
from sklearn.utils import shuffle
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import RandomOverSampler

### 20. Load file lên và shuffle nó

In [3]:
data = pd.read_csv("./data/standard_reviews.csv")

In [4]:
data = shuffle(data)

In [5]:
data

,raw_comment,standard_comment,label
20177,"Giao nhanh, đóng gói kĩ, chất lượng ok, săn sa...",giá_rẻ đáng_mua giao_nhanh đóng gói chất_lượng...,1
197502,"Chất vải mặc khá nóng, form lên cũng vài chỗ k ổn",không_ổn chất_vải mặc lên cũng,0
91317,Đặt cùng Size nhưng mẫu này size lớn so với nh...,đặt nhưng này với nên shop khi nha người,1
131141,Áo khá đẹp. Chất lượng cũng k tệ. Giao hàng hơ...,tệ khá_đẹp áo chất_lượng cũng không giao hàng,1
140468,"Áo đẹp không có gì để chê, size vừa vặn, shop ...",áo_đẹp vừa_vặn nên_mua rất_đáng đáng_tiền khôn...,1
...,...,...,...
91109,"Đẹp cực luôn nma hơi rộng, ship bị lâu.",lâu đẹp_cực hơi_rộng luôn nhưng mà bị,1
37599,Dây chuyền rất xinh. Nhỏ nhỏ rất đáng yêu mà v...,rất_đáng rất mà vẫn chất nhé,1
39550,Đã nhận hàng. Ví nhỏ nhỏ xinh xinh nhưng m ở x...,nhận_hàng đã nhưng ở nên,1
140061,Hình ảnh chỉ mang tính chất nhận xu do mình lư...,như_hình hơi_mỏng chỉ chất mình hàng quần có n...,1


### 21. Kiểm tra số mẫu giữa các nhóm

In [6]:
no_samples_each_group = data[['label', 'standard_comment']].groupby(['label']).count()

In [7]:
no_samples_each_group

,standard_comment
label,
-1,3239
0,11014
1,187102


> **Nhận xét**:
> * Số lượng mẫu giữa các group không đều.
>
> $\Rightarrow$ thực hiện oversampling

### 22. Tạo tập `final_data`

In [8]:
no_samples_each_group = no_samples_each_group['standard_comment'].to_list()

In [9]:
negative = data.label[data.label.eq(-1)].sample(no_samples_each_group[0]).index
normal = data.label[data.label.eq(0)].sample(no_samples_each_group[1]).index
positive = data.label[data.label.eq(1)].sample(no_samples_each_group[1]).index

final_data = data.loc[negative.union(normal).union(positive)]

In [10]:
final_data = shuffle(final_data)

In [11]:
final_data

,raw_comment,standard_comment,label
174442,Chất lượng lắm ạ :3 có hộp đàng hoàng luônnn ạ,chất_lượng lắm ạ có luôn ạ,1
102056,"Sản phẩm đúng mẫu mã kiểu dáng, mẫu mã, đóng h...",cẩn_thận sản_phẩm đóng hàng và giao hàng,1
19187,Shop giao đủ\nTuy nhiên sản phẩm không giống h...,giao_đủ không_cho không_giống_hình shop sản_ph...,0
196050,"Quần mua với giá 139k, mình 1m78 63kg mặc size...",chật tạm_ổn cũng_tạm hơi_mỏng không_mặc quần m...,0
158499,Hàng đep giá cũng rẻ giao hàng nhanh mình rất ...,ủng_hộ hàng_đẹp rất_hài_lòng giao_hàng_nhanh c...,1
...,...,...,...
125529,"Mình đặt mẫu with diamond cỡ 6 , mà giao nhẫn ...",cũng_được mình đặt mà giao mà thôi,-1
25818,"Dễ hỏng, vừa mua đã hỏng 1 cái!!!!!!!!!!!!!!!!!!!",mua đã cái,-1
137986,"K đẹp lắm, chất len bai dão",đẹp_lắm không_đẹp không chất,0
183108,Shop gửi hàng nhanh. Đóng gói đơn hàng cẩn thậ...,cẩn_thận shop hàng đóng_gói hàng sản_phẩm ok,1


In [12]:
final_data = final_data.reset_index(drop=True)

### 23. Thực hiện TF-IDF với `ngram_range=(1, 4)`

In [13]:
tf_idf = TfidfVectorizer(sublinear_tf=True, min_df=5, norm='l2', encoding='utf-8', ngram_range=(1, 4))
tf_idf.fit(final_data['standard_comment'])

TfidfVectorizer(min_df=5, ngram_range=(1, 4), sublinear_tf=True)

### 24. Tạo tập input `X` và output `y`

In [14]:
X = tf_idf.transform(final_data['standard_comment'])
y = final_data['label']

### 25. Thực hiện over-sampling

In [15]:
ros = RandomOverSampler()
X_ros, y_ros = ros.fit_resample(X, y)

In [16]:
Counter(y_ros)

Counter({1: 11014, 0: 11014, -1: 11014})

### 26. Train test split

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X_ros, y_ros, test_size=0.2, random_state=100, shuffle=True)

### 27. Lưu lại `final_data`, `tf_idf`, `X_train`, `X_test`, `y_train`, `y_test`

In [18]:
final_data.to_csv("./data/final_data.csv", index=False)

In [19]:
pickle.dump(tf_idf, open("./data/tf_idf.pickle", "wb"))
pickle.dump(X_train, open("./data/X_train.pickle", "wb"))
pickle.dump(X_test, open("./data/X_test.pickle", "wb"))
pickle.dump(y_train, open("./data/y_train.pickle", "wb"))
pickle.dump(y_test, open("./data/y_test.pickle", "wb"))